# Müzik Türü Sınıflandırma Projesi

Bu notebook, FMA (Free Music Archive) veri setini kullanarak müzik türü sınıflandırma modeli geliştirmek için veri hazırlama ve dengeleme işlemlerini içermektedir.

## Gerekli Kütüphanelerin İçe Aktarılması
Aşağıdaki hücrede, projede kullanılacak temel Python kütüphaneleri import edilmektedir:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import RandomOverSampler, BorderlineSMOTE
from collections import Counter

%matplotlib inline
sns.set(style='whitegrid')

## Yardımcı Fonksiyonlar

### Sınıf Dağılımı Görselleştirme Fonksiyonu
Aşağıdaki fonksiyon, veri setindeki sınıf dağılımlarını görselleştirmek için kullanılacaktır. Bu görselleştirme, veri dengesizliğini anlamamıza yardımcı olur.

In [ ]:
def plot_class_distribution(y, labels, title):
    counts = pd.Series(y).value_counts().sort_index()
    valid_indices = counts.index[counts.index < len(labels)]
    counts = counts.loc[valid_indices]
    names = labels[counts.index]

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(x=names, y=counts.values, hue=names, palette='viridis', legend=False)
    ax.set_title(title)
    ax.set_xlabel('Sınıf')
    ax.set_ylabel('Sayı')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Veri Yükleme ve Ön İşleme

Bu bölümdeki fonksiyon:
- FMA metadata dosyalarını yükler
- Gerekli sütunları seçer
- Eksik verileri temizler
- Etiketleri kodlar
- Veriyi sayısal formata dönüştürür

In [ ]:
def load_data():
    tracks_path = 'fma_metadata/tracks.csv'
    features_path = 'fma_metadata/features.csv'

    if not os.path.exists(tracks_path) or not os.path.exists(features_path):
        raise FileNotFoundError(f"Gerekli veri dosyaları bulunamadı. '{tracks_path}' ve '{features_path}' dosyalarının mevcut olduğundan emin olun.")

    tracks = pd.read_csv(tracks_path, index_col=0, header=[0,1])
    
    features = pd.read_csv(features_path, index_col=0, header=[0,1])  # Çok seviyeli başlıkla oku
    features = features.loc[:, features.columns.get_level_values(0) != 'statistics']  # 'statistics' sütunlarını kaldır
    features = features.astype(np.float32)  # Sayısal olmayan sütunları kaldırdıktan sonra float'a dönüştür

    features.index = features.index.astype(str)
    tracks.index = tracks.index.astype(str)

    genre_series = tracks[('track', 'genre_top')].dropna()
    common_index = features.index.intersection(genre_series.index)

    X = features.loc[common_index]
    y_labels = genre_series.loc[common_index]

    X = X.fillna(0).replace([np.inf, -np.inf], 0).astype(np.float32)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y_labels)

    print('Veriler yüklendi ve önişlendi.')
    return X, y, label_encoder

## Başlangıç Veri Analizi

Verinin ilk yüklemesini yapıp, başlangıçtaki sınıf dağılımını inceleyelim. Bu analiz, veri dengesizliği problemini görselleştirmemize yardımcı olacak.

In [ ]:
# Veriyi yükle ve önişle
X, y, le = load_data()

# Başlangıç dağılımını göster
plot_class_distribution(y, le.classes_, 'Başlangıç Sınıf Dağılımı')

## Veri Bölme ve Eğitim Seti Analizi

Veriyi eğitim ve test setlerine ayırıp, eğitim setindeki sınıf dağılımını inceliyoruz. Stratified split kullanarak orijinal dağılımı koruyoruz.

In [ ]:
# Veriyi böl ve eğitim dağılımını göster
# İlk bölme: Ana eğitim ve test setleri
X_train_orig, X_test, y_train_orig, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# İkinci bölme: Ana eğitim setini resampling ve temiz doğrulama setlerine ayır
X_train_for_resample, X_val_clean, y_train_for_resample, y_val_clean = train_test_split(
    X_train_orig, y_train_orig, test_size=0.15, stratify=y_train_orig, random_state=42
)

print(f'İlk bölünme tamamlandı: X_train_orig {X_train_orig.shape}, X_test {X_test.shape}')
print(f'İkinci bölünme tamamlandı: X_train_for_resample {X_train_for_resample.shape}, X_val_clean {X_val_clean.shape}')

plot_class_distribution(y_train_for_resample, le.classes_, 'Resampling İçin Eğitim Seti Dağılımı')

# Detaylı dağılımı yazdır
unique, counts = np.unique(y_train_for_resample, return_counts=True)
print("\nResampling İçin Eğitim Seti Dağılımı (ham sayılar):")
for i, (u, c) in enumerate(zip(unique, counts)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

## Veri Dengeleme - Aşama 1

İlk aşamada, çok az örneğe sahip sınıflar için RandomOverSampler kullanılıyor. Bu aşama, BorderlineSMOTE için yeterli örnek sayısına ulaşmamızı sağlar.

In [ ]:
# Enhanced Oversampling Strategy - MORE AGGRESSIVE FOR BETTER BALANCE
print('\nEnhanced oversampling strategy for better class balance...')

# Get current distribution
unique, counts = np.unique(y_train_for_resample, return_counts=True)
print(f"Original distribution - Max: {max(counts)}, Min: {min(counts)}, Classes: {len(unique)}")

# More aggressive strategy: bring all classes to at least 25% of the max class
max_samples = max(counts)
target_min = max(15, int(max_samples * 0.25))  # At least 15 samples or 25% of max class

print(f"Target minimum samples per class: {target_min}")

# Create sampling strategy
sampling_strategy = {}
total_added = 0

for class_idx, count in zip(unique, counts):
    if count < target_min:
        sampling_strategy[class_idx] = target_min
        total_added += (target_min - count)
        print(f"  Boosting {le.classes_[class_idx]}: {count} → {target_min} (+{target_min - count})")

if sampling_strategy:
    print(f"\nApplying oversampling to {len(sampling_strategy)} classes...")
    print(f"Total samples to be added: {total_added}")
    
    ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
    X_res, y_res = ros.fit_resample(X_train_for_resample, y_train_for_resample)
    
    print(f"✅ Enhanced oversampling completed!")
    print(f"Dataset size: {len(y_train_for_resample)} → {len(y_res)} (+{len(y_res) - len(y_train_for_resample)})")
else:
    print("No oversampling needed - all classes already well represented")
    X_res, y_res = X_train_for_resample, y_train_for_resample

# Show final distribution
unique_final, counts_final = np.unique(y_res, return_counts=True)
print(f"\nFinal distribution after enhanced oversampling:")
print(f"Max: {max(counts_final)}, Min: {min(counts_final)}, Ratio: {max(counts_final)/min(counts_final):.2f}")

for i, (u, c) in enumerate(zip(unique_final, counts_final)):
    if i < len(le.classes_):
        print(f"  {le.classes_[u]}: {c} samples")

plot_class_distribution(y_res, le.classes_, 'Enhanced Oversampling Results')

## Veri Dengeleme - Aşama 2

İkinci aşamada, daha sofistike bir yaklaşım olan BorderlineSMOTE kullanılarak kalan sınıflar dengeleniyor. Bu yöntem, sadece rastgele kopyalama yerine sentetik örnekler oluşturur.

Not: Bu aşama, veri setinin yapısına bağlı olarak başarısız olabilir. Bu durumda, ilk aşamadaki sonuçlar kullanılacaktır.

In [ ]:
# REMOVE BorderlineSMOTE - Major overfitting culprit
print('\nSkipping BorderlineSMOTE to prevent overfitting...')
print('Using only minimal RandomOverSampler results for better generalization.')

# No BorderlineSMOTE - use minimal oversampling results directly
print("\nFinal distribution after minimal oversampling:")
unique_final, counts_final = np.unique(y_res, return_counts=True)
for i, (u, c) in enumerate(zip(unique_final, counts_final)):
    print(f"Sınıf {u} ({le.classes_[i]}): {c} örnek")

print("\nOversampling pipeline completed. Using conservative approach for better generalization.")

## Sınıf Ağırlıklandırma (Class Weighting)

Veri dengelemeye ek olarak, model eğitimi sırasında sınıf ağırlıklandırma uygulayacağız. Bu yaklaşım, az temsil edilen sınıflara daha fazla önem vererek modelin bu sınıfları daha iyi öğrenmesini sağlar.

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Simplified class weighting - focus on original imbalance
print("\nCalculating class weights for loss function...")

# Calculate weights from original unbalanced data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_for_resample),
    y=y_train_for_resample
)

# Convert to PyTorch tensor
class_weights_tensor = torch.FloatTensor(class_weights)

print("Class weights calculated:")
for i, weight in enumerate(class_weights):
    if i < len(le.classes_):
        print(f"{le.classes_[i]}: {weight:.3f}")

print(f"\nClass weights tensor shape: {class_weights_tensor.shape}")
print(f"Max weight: {class_weights_tensor.max():.3f}, Min weight: {class_weights_tensor.min():.3f}")

# Check if weights are too extreme
weight_ratio = class_weights_tensor.max() / class_weights_tensor.min()
print(f"Weight ratio (max/min): {weight_ratio:.2f}")
if weight_ratio > 10:
    print("⚠️  WARNING: Very extreme class weights detected!")
    print("This may cause training instability. Consider using no weights or softer weights.")
    
    # Option to use no class weights
    USE_CLASS_WEIGHTS = False  # Set to False if weights are too extreme
    if not USE_CLASS_WEIGHTS:
        print("Disabling class weights for stability...")
        class_weights_tensor = torch.ones(len(le.classes_))
else:
    print("✅ Class weights look reasonable.")
    USE_CLASS_WEIGHTS = True

print("These weights will be used in CrossEntropyLoss.")
print(f"Using class weights: {USE_CLASS_WEIGHTS}")

In [ ]:
# DETAILED DATA FLOW VERIFICATION
print("\n=== DETAILED DATA PREPARATION SUMMARY ===")
print(f"Original training samples: {len(y_train_for_resample)}")
print(f"After minimal oversampling: {len(y_res)}")
print(f"Features: {X_res.shape[1]}")
print(f"Classes: {len(le.classes_)}")

# Show detailed class distribution comparison
print("\n🔍 OVERSAMPLING VERIFICATION:")
print("BEFORE oversampling:")
unique_before, counts_before = np.unique(y_train_for_resample, return_counts=True)
for i, (u, c) in enumerate(zip(unique_before, counts_before)):
    if i < len(le.classes_):
        print(f"  {le.classes_[u]}: {c} samples")

print("\nAFTER oversampling:")
unique_after, counts_after = np.unique(y_res, return_counts=True)
for i, (u, c) in enumerate(zip(unique_after, counts_after)):
    if i < len(le.classes_):
        print(f"  {le.classes_[u]}: {c} samples")

# Calculate oversampling effect
total_before = len(y_train_for_resample)
total_after = len(y_res)
oversampling_factor = total_after / total_before
print(f"\nOversampling effect: {total_before} → {total_after} samples")
print(f"Oversampling factor: {oversampling_factor:.2f}x")

if oversampling_factor <= 1.05:
    print("⚠️  WARNING: Very little or no oversampling applied!")
    print("This might explain why accuracy hasn't improved significantly.")
else:
    print("✅ Oversampling successfully applied!")

print("\nProceeding to feature selection and model training...")

## Özellik Seçimi (K-Best Feature Selection)

Model performansını artırmak ve aşırı öğrenmeyi (overfitting) azaltmak için K-Best özellik seçimi algoritmasını uygulayacağız. Bu algoritma, her özelliğin hedef değişkenle olan istatistiksel ilişkisini ölçer ve en anlamlı K özelliği seçer.

In [ ]:
# K-Best özellik seçimi uygulaması - OPTIMIZED FOR BETTER PERFORMANCE
print('\nK-Best özellik seçimi uygulanıyor...')

k = 150  # INCREASED from 100 - more features for better performance
print(f"Toplam özellik sayısı: {X_res.shape[1]}, Seçilecek özellik sayısı: {k}")

# SelectKBest ile özellik seçimi - Sadece resampled data üzerinde fit et
selector = SelectKBest(score_func=f_classif, k=k)
X_res_selected = selector.fit_transform(X_res, y_res)

# Fitted selector ile validation ve test setlerini transform et
X_val_clean_selected = selector.transform(X_val_clean)
X_test_selected = selector.transform(X_test)

print(f"Özellik seçimi tamamlandı. Seçilen özelliklerin boyutu: {X_res_selected.shape}")
print(f"Validation set boyutu: {X_val_clean_selected.shape}")
print(f"Test set boyutu: {X_test_selected.shape}")

# Orijinal veriyi güncellenmiş veri ile değiştirelim
X_res = X_res_selected
X_val_clean = X_val_clean_selected
X_test = X_test_selected

# Veri Ölçeklendirme (RobustScaler) - Better for outliers
print('\nVeri ölçeklendirme uygulanıyor (RobustScaler)...')
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()  # More robust to outliers than StandardScaler
X_res_scaled = scaler.fit_transform(X_res)
X_val_clean_scaled = scaler.transform(X_val_clean)
X_test_scaled = scaler.transform(X_test)

print("Veri ölçeklendirme tamamlandı.")
print(f"Ölçeklenmiş resampled eğitim verisi boyutu: {X_res_scaled.shape}")
print(f"Ölçeklenmiş validation verisi boyutu: {X_val_clean_scaled.shape}")
print(f"Ölçeklenmiş test verisi boyutu: {X_test_scaled.shape}")

# Veriyi güncellenmiş ölçeklenmiş veriler ile değiştir
X_res = X_res_scaled
X_val_clean = X_val_clean_scaled
X_test = X_test_scaled

*-----------------------------------------------------------------------------------*
# PyTorch LSTM MODEL EĞİTİMİ
*-----------------------------------------------------------------------------------*

In [ ]:
print("\nPyTorch LSTM Model Eğitimi Başlıyor...")

# Veri yükleme, önişleme, bölme ve dengeleme adımlarının tamamlandığı varsayılır.
# Bu noktada aşağıdaki değişkenlerin mevcut olması beklenir:
# X_res, y_res (Dengelenmiş eğitim verisi)
# X_val, y_val (Doğrulama verisi)
# X_test, y_test (Test verisi)
# le (LabelEncoder nesnesi)

## Gelişmiş Özellik Mühendisliği ve Model Optimizasyonu (İsteğe Bağlı)

Bu bölümde, model performansını artırmak için gelişmiş özellik mühendisliği tekniklerini ve model optimizasyonlarını güvenli bir şekilde uygulayabiliriz. Bu teknikler veri sızıntısını önlemek için dikkatli bir şekilde tasarlanmıştır.

In [ ]:
# Enhanced feature engineering - DISABLED to prevent overfitting
print("Enhanced feature engineering is DISABLED to prevent overfitting.")
print("Using only basic features with minimal oversampling approach.")

print(f"\nFinal data dimensions:")
print(f"Training (X_res): {X_res.shape}")
print(f"Validation (X_val_clean): {X_val_clean.shape}")
print(f"Test (X_test): {X_test.shape}")

print("\nSimple and conservative approach for better generalization.")

## LSTM Modeli için Veri Hazırlığı

PyTorch LSTM modeli için, veriyi uygun formata dönüştürmemiz gerekir. LSTM modeller sıralı veri bekler, bu nedenle öznitelik vektörünü zamansal bir diziye dönüştüreceğiz.

In [ ]:
# PyTorch tensörlerine dönüştürme ve veri setlerini hazırlama
def create_sequence_data(X, y, sequence_length=10):
    """
    Öznitelik vektörünü sıralı verilere dönüştürür.
    FMA veri seti sıralı yapıda değil, bu nedenle yapay bir sıra oluşturuyoruz.
    """
    print(f"Creating sequences from {X.shape[0]} samples with {X.shape[1]} features...")
    
    # Veri boyutlarını kontrol et
    n_samples, n_features = X.shape
    
    # Veriyi yeniden şekillendirme
    features_per_timestep = n_features // sequence_length
    
    if features_per_timestep == 0:
        features_per_timestep = 1
        sequence_length = min(sequence_length, n_features)
    
    print(f"Sequence config: {sequence_length} timesteps, {features_per_timestep} features per timestep")
    
    # Son timestep'e sığmayan özellikleri ele alma
    remainder = n_features - (sequence_length * features_per_timestep)
    if remainder > 0:
        print(f"Note: {remainder} features will be unused due to sequence reshaping")
    
    # Yeniden şekillendirilmiş veri için array oluşturma
    X_seq = np.zeros((n_samples, sequence_length, features_per_timestep))
    
    # Veriyi yeniden şekillendirme
    for i in range(n_samples):
        for t in range(sequence_length):
            start_idx = t * features_per_timestep
            end_idx = min(start_idx + features_per_timestep, n_features)
            
            if start_idx < n_features:
                X_seq[i, t, :end_idx-start_idx] = X[i, start_idx:end_idx]
    
    # PyTorch tensörlerine dönüştürme
    X_tensor = torch.FloatTensor(X_seq)
    y_tensor = torch.LongTensor(y)
    
    print(f"✅ Sequence creation completed: {X_tensor.shape} -> {y_tensor.shape}")
    return X_tensor, y_tensor

# Optimized sequence parameters
sequence_length = 10  # INCREASED from 5 for better temporal modeling

print("\n=== CREATING SEQUENCE DATA ===")
print("Converting scaled data to sequence format...")

# Ölçeklenmiş verileri sıralı forma dönüştürme
X_train_seq, y_train_tensor = create_sequence_data(X_res, y_res, sequence_length)
X_val_seq, y_val_tensor = create_sequence_data(X_val_clean, y_val_clean, sequence_length)
X_test_seq, y_test_tensor = create_sequence_data(X_test, y_test, sequence_length)

print(f"\nFinal data shapes:")
print(f"  Training: {X_train_seq.shape} samples")
print(f"  Validation: {X_val_seq.shape} samples")
print(f"  Test: {X_test_seq.shape} samples")

# Optimized DataLoader settings
batch_size = 256  # REDUCED from 512 for better gradient updates
train_dataset = TensorDataset(X_train_seq, y_train_tensor)
val_dataset = TensorDataset(X_val_seq, y_val_tensor)
test_dataset = TensorDataset(X_test_seq, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"\nDataLoaders created:")
print(f"  Training batches: {len(train_loader)} (batch_size={batch_size})")
print(f"  Validation batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")
print(f"  Total training samples: {len(train_loader.dataset)}")

## LSTM Model Tanımı ve Eğitimi

Aşağıda müzik türü sınıflandırması için bir LSTM (Long Short-Term Memory) ağı tanımlıyoruz. LSTM'ler, müzik gibi sıralı verilerde başarılı olan bir derin öğrenme mimarisidir.

In [ ]:
# Enhanced LSTM model - FOCUSED IMPROVEMENTS FOR BETTER ACCURACY
class MusicGenreLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(MusicGenreLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Bidirectional LSTM for better feature capture
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True  # Key improvement: bidirectional
        )
        
        # Batch normalization for training stability
        self.batch_norm = nn.BatchNorm1d(hidden_size * 2)  # *2 for bidirectional
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)
        
        # Two-layer classifier for better decision boundary
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)  # *2 for bidirectional
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # LSTM forward pass
        lstm_out, _ = self.lstm(x)
        
        # Use last time step output (both directions)
        lstm_out = lstm_out[:, -1, :]
        
        # Batch normalization
        lstm_out = self.batch_norm(lstm_out)
        
        # First FC layer with activation and dropout
        out = self.fc1(lstm_out)
        out = self.relu(out)
        out = self.dropout(out)
        
        # Final classification layer
        out = self.fc2(out)
        
        return out

# Improved model parameters - balanced for better performance
input_size = X_train_seq.shape[2]
hidden_size = 64  # Keep balanced
num_layers = 2  # Increased for more capacity
num_classes = len(le.classes_)
dropout = 0.3  # Optimal dropout

# GPU kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Enhanced model creation
model = MusicGenreLSTM(input_size, hidden_size, num_layers, num_classes, dropout).to(device)
print(model)

# Model parameter count
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params:,}")

# Enhanced loss function and optimizer
print(f"Class weights being used: {class_weights_tensor}")
class_weights_device = class_weights_tensor.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_device, label_smoothing=0.1)  # Label smoothing
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)  # AdamW for better generalization
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, eta_min=1e-6)  # Better LR schedule

# Enhanced Training function - FOCUSED IMPROVEMENTS
def train_model_enhanced(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    best_val_acc = 0.0  # Track best validation accuracy instead of loss
    best_model = None
    patience_counter = 0
    PATIENCE = 7  # Increased patience for better convergence
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping for stability
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_train_acc = train_correct / train_total
        epoch_val_acc = val_correct / val_total
        
        # Learning rate scheduling
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store metrics
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accs.append(epoch_train_acc)
        val_accs.append(epoch_val_acc)
        
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, '
              f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}, LR: {current_lr:.6f}')
        
        # Save best model based on validation accuracy
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_model = model.state_dict().copy()
            patience_counter = 0
            print(f'*** New best validation accuracy: {best_val_acc:.4f} ***')
        else:
            patience_counter += 1
            
        if patience_counter >= PATIENCE:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    # Load best model
    model.load_state_dict(best_model)
    print(f'Best validation accuracy achieved: {best_val_acc:.4f}')
    return model, train_losses, val_losses, train_accs, val_accs

# Train with enhanced settings
print("Training with enhanced settings for better accuracy...")
model, train_losses, val_losses, train_accs, val_accs = train_model_enhanced(
    model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25
)
print("Enhanced training completed!")

In [ ]:
# DIAGNOSTIC ANALYSIS - Check for potential issues
print("=== DIAGNOSTIC ANALYSIS ===")

# 1. Check data shapes and ranges
print(f"Training data shape: {X_train_seq.shape}")
print(f"Training labels shape: {y_train_tensor.shape}")
print(f"Unique classes in training: {torch.unique(y_train_tensor)}")
print(f"Data range - Min: {X_train_seq.min():.4f}, Max: {X_train_seq.max():.4f}")
print(f"Data mean: {X_train_seq.mean():.4f}, std: {X_train_seq.std():.4f}")

# 2. Check class distribution
class_counts = torch.bincount(y_train_tensor)
print(f"\nClass distribution in training:")
for i, count in enumerate(class_counts):
    if i < len(le.classes_):
        print(f"  {le.classes_[i]}: {count} samples")

# 3. Check for data leakage or issues
print(f"\nValidation data shape: {X_val_seq.shape}")
print(f"Test data shape: {X_test_seq.shape}")
print(f"Val data range - Min: {X_val_seq.min():.4f}, Max: {X_val_seq.max():.4f}")

# 4. Check model complexity
model_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
data_samples = len(y_train_tensor)
params_per_sample = model_params / data_samples
print(f"\nModel complexity analysis:")
print(f"  Total parameters: {model_params:,}")
print(f"  Training samples: {data_samples:,}")
print(f"  Parameters per sample: {params_per_sample:.2f}")
if params_per_sample > 10:
    print("  ⚠️  WARNING: High parameter-to-sample ratio may cause overfitting")
elif params_per_sample < 0.1:
    print("  ⚠️  WARNING: Very low parameter-to-sample ratio may cause underfitting")
else:
    print("  ✅ Parameter-to-sample ratio looks reasonable")

# 5. Quick baseline test
print(f"\nBaseline accuracy (most frequent class): {class_counts.max().item() / data_samples:.4f}")
print("If model performs worse than this, there's a serious issue.")

print("\n" + "="*50)

In [ ]:
# SIMPLE BASELINE COMPARISON
print("=== BASELINE MODEL COMPARISON ===")

# Let's try a simple feedforward network first to establish baseline
class SimpleFFN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout=0.3):
        super(SimpleFFN, self).__init__()
        # Flatten the sequence dimension
        self.flatten_size = input_size * 5  # sequence_length = 5
        
        self.layers = nn.Sequential(
            nn.Linear(self.flatten_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )
    
    def forward(self, x):
        # Flatten sequence data
        x = x.view(x.size(0), -1)
        return self.layers(x)

# Create baseline model
baseline_model = SimpleFFN(input_size, 64, num_classes, 0.3).to(device)
baseline_params = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)
print(f"Baseline FFN parameters: {baseline_params:,}")
print(f"LSTM parameters: {model_params:,}")

# Quick test to see if data flows correctly
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    sample_input, sample_labels = sample_batch[0][:5].to(device), sample_batch[1][:5].to(device)
    
    # Test LSTM
    lstm_output = model(sample_input)
    print(f"\nLSTM output shape: {lstm_output.shape}")
    print(f"LSTM output range: {lstm_output.min():.4f} to {lstm_output.max():.4f}")
    
    # Test FFN
    ffn_output = baseline_model(sample_input)
    print(f"FFN output shape: {ffn_output.shape}")
    print(f"FFN output range: {ffn_output.min():.4f} to {ffn_output.max():.4f}")
    
    print(f"Sample labels: {sample_labels}")
    print(f"Expected output classes: {num_classes}")

print("\nModels initialized successfully. Proceeding with training...")
print("=" * 50)

In [ ]:
# ENSEMBLE APPROACH FOR BETTER ACCURACY
print("=== ENSEMBLE TRAINING ===")

# Train multiple models with different initialization
class EnsembleModel:
    def __init__(self, num_models=3):
        self.models = []
        self.num_models = num_models
        
        for i in range(num_models):
            model = MusicGenreLSTM(input_size, hidden_size, num_layers, num_classes, dropout).to(device)
            self.models.append(model)
            print(f"Model {i+1} initialized")
    
    def train_ensemble(self, train_loader, val_loader, criterion, num_epochs=20):
        """Train all models in ensemble"""
        trained_models = []
        
        for i, model in enumerate(self.models):
            print(f"\nTraining Model {i+1}/{self.num_models}...")
            
            # Create separate optimizer for each model
            optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
            scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)
            
            # Train model
            trained_model, _, _, _, _ = train_model_enhanced(
                model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs
            )
            trained_models.append(trained_model)
        
        self.models = trained_models
        return self.models
    
    def predict(self, data_loader):
        """Make ensemble predictions"""
        all_predictions = []
        
        # Get predictions from each model
        for model in self.models:
            model.eval()
            predictions = []
            
            with torch.no_grad():
                for inputs, _ in data_loader:
                    inputs = inputs.to(device)
                    outputs = model(inputs)
                    probabilities = torch.softmax(outputs, dim=1)
                    predictions.append(probabilities.cpu())
            
            all_predictions.append(torch.cat(predictions, dim=0))
        
        # Average predictions
        ensemble_predictions = torch.stack(all_predictions).mean(dim=0)
        return ensemble_predictions

# Create and train ensemble (using 3 models for balance of performance and time)
ensemble = EnsembleModel(num_models=3)
print(f"Training ensemble of {ensemble.num_models} models...")

In [ ]:
# DATA AUGMENTATION FOR BETTER GENERALIZATION
print("\n=== DATA AUGMENTATION ===")

def augment_data(X, y, augment_factor=0.2):
    """Simple data augmentation with noise injection"""
    n_samples = X.shape[0]
    n_augmented = int(n_samples * augment_factor)
    
    # Random indices for augmentation
    aug_indices = np.random.choice(n_samples, n_augmented, replace=True)
    
    # Create augmented samples
    X_aug = X[aug_indices].copy()
    y_aug = y[aug_indices].copy()
    
    # Add small gaussian noise
    noise_std = 0.01 * X_aug.std()
    noise = np.random.normal(0, noise_std, X_aug.shape)
    X_aug = X_aug + noise
    
    # Combine original and augmented data
    X_combined = np.vstack([X, X_aug])
    y_combined = np.hstack([y, y_aug])
    
    return X_combined, y_combined

# Apply data augmentation to training data
print("Applying data augmentation...")
X_res_aug, y_res_aug = augment_data(X_res, y_res, augment_factor=0.15)

print(f"Original training size: {X_res.shape[0]}")
print(f"Augmented training size: {X_res_aug.shape[0]}")
print(f"Augmentation factor: {(X_res_aug.shape[0] - X_res.shape[0]) / X_res.shape[0]:.2f}")

# CRITICAL: Verify class distribution is maintained after augmentation
print("\n🔍 VERIFYING AUGMENTATION EFFECT:")
print("Before augmentation:")
unique_before_aug, counts_before_aug = np.unique(y_res, return_counts=True)
for i, (u, c) in enumerate(zip(unique_before_aug, counts_before_aug)):
    if i < len(le.classes_):
        print(f"  {le.classes_[u]}: {c} samples")

print("\nAfter augmentation:")
unique_after_aug, counts_after_aug = np.unique(y_res_aug, return_counts=True)
for i, (u, c) in enumerate(zip(unique_after_aug, counts_after_aug)):
    if i < len(le.classes_):
        print(f"  {le.classes_[u]}: {c} samples")

# Update training data
X_res = X_res_aug
y_res = y_res_aug

# Recreate sequence data with augmented dataset
X_train_seq, y_train_tensor = create_sequence_data(X_res, y_res, sequence_length)
print(f"\nNew training sequence shape: {X_train_seq.shape}")
print(f"Final training labels: {len(y_train_tensor)} samples")

# Update train loader
train_dataset = TensorDataset(X_train_seq, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

print("✅ Data augmentation completed successfully!")
print(f"Final DataLoader has {len(train_loader)} batches with batch_size={batch_size}")
print(f"Total training samples in DataLoader: {len(train_loader.dataset)}")

In [ ]:
# CRITICAL: VERIFY DATA PIPELINE INTEGRITY
print("\n=== DATA PIPELINE VERIFICATION ===")
print("Checking if our oversampling and augmentation actually made it to the training loop...")

# Check actual class distribution in the final training tensor
final_class_counts = torch.bincount(y_train_tensor)
print("\n📊 FINAL TRAINING DATA DISTRIBUTION:")
for i, count in enumerate(final_class_counts):
    if i < len(le.classes_):
        print(f"  {le.classes_[i]}: {count} samples")

# Calculate balance metrics
max_count = final_class_counts.max().item()
min_count = final_class_counts.min().item()
balance_ratio = max_count / min_count if min_count > 0 else float('inf')

print(f"\nBalance Analysis:")
print(f"  Most samples: {max_count}")
print(f"  Least samples: {min_count}")
print(f"  Imbalance ratio: {balance_ratio:.2f}")

if balance_ratio > 10:
    print("  ⚠️  WARNING: Still very imbalanced! Oversampling may not have worked.")
elif balance_ratio > 5:
    print("  📊 Moderate imbalance - this is acceptable")
else:
    print("  ✅ Good balance achieved!")

# Verify data actually changed from original
original_samples = len(y_train_for_resample)
final_samples = len(y_train_tensor)
data_increase = (final_samples - original_samples) / original_samples * 100

print(f"\nData Growth Verification:")
print(f"  Original samples: {original_samples}")
print(f"  Final samples: {final_samples}")
print(f"  Increase: {data_increase:.1f}%")

if data_increase < 5:
    print("  ⚠️  WARNING: Very little data augmentation applied!")
    print("  This suggests oversampling/augmentation isn't working properly.")
else:
    print(f"  ✅ Data successfully augmented by {data_increase:.1f}%")

print("\n" + "="*50)

## Model Değerlendirmesi ve Görselleştirme

Bu bölümde eğitilmiş modeli test veri seti üzerinde değerlendirip, sonuçları görselleştireceğiz.

In [ ]:
# Eğitim sonuçlarını görselleştirme
def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(14, 5))
    
    # Kayıp grafiği
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Eğitim', marker='o')
    plt.plot(val_losses, label='Doğrulama', marker='*')
    plt.title('Model Kaybı')
    plt.xlabel('Epoch')
    plt.ylabel('Kayıp (Cross-Entropy)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Doğruluk grafiği
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Eğitim', marker='o')
    plt.plot(val_accs, label='Doğrulama', marker='*')
    plt.title('Model Doğruluğu')
    plt.xlabel('Epoch')
    plt.ylabel('Doğruluk')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Eğitim sonuçlarını görselleştir
try:
    plot_training_history(train_losses, val_losses, train_accs, val_accs)
except NameError:
    print("Eğitim geçmişi bulunamadı. Önce modeli eğitin.")

# Test veri seti üzerinde değerlendirme
def evaluate_model(model, test_loader, device):
    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    
    # Doğruluk hesapla
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    # Sonuçları yazdır
    print(f"Test Doğruluğu: {accuracy:.4f}")
    
    # Sınıflandırma raporu
    print("\nSınıflandırma Raporu:")
    print(classification_report(y_true, y_pred, target_names=le.classes_, 
                               zero_division=0, labels=np.unique(y_true)))
    
    # Class-wise performance analysis
    print("\nDetaylı Sınıf Bazında Performans:")
    class_report = classification_report(y_true, y_pred, target_names=le.classes_, 
                                       output_dict=True, zero_division=0, labels=np.unique(y_true))
    for class_name, metrics in class_report.items():
        if isinstance(metrics, dict) and 'f1-score' in metrics:
            print(f"{class_name}: Precision={metrics['precision']:.3f}, Recall={metrics['recall']:.3f}, F1={metrics['f1-score']:.3f}, Support={metrics['support']}")
    
    # Karmaşıklık matrisi
    plt.figure(figsize=(12, 10))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title('Karmaşıklık Matrisi')
    plt.xlabel('Tahmin Edilen Etiketler')
    plt.ylabel('Gerçek Etiketler')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    return y_true, y_pred

# Test veri seti üzerinde değerlendir
try:
    y_true, y_pred = evaluate_model(model, test_loader, device)
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
import torch.nn.functional as F

def get_model_probabilities(model, test_loader, device):
    """
    Get prediction probabilities from the trained model
    """
    model.eval()
    y_true = []
    y_proba = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            
            y_true.extend(labels.cpu().numpy())
            y_proba.extend(probabilities.cpu().numpy())
    
    return np.array(y_true), np.array(y_proba)

def comprehensive_evaluation(model, test_loader, device, class_names):
    """
    Provide detailed evaluation metrics for the trained model
    """
    # Get true labels and prediction probabilities
    y_true, y_proba = get_model_probabilities(model, test_loader, device)
    
    # Get predictions
    y_pred = np.argmax(y_proba, axis=1)
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, average=None, labels=range(len(class_names))
    )
    
    # Macro and weighted averages
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro'
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    # AUC-ROC for multiclass
    y_true_binarized = label_binarize(y_true, classes=range(len(class_names)))
    auc_scores = []
    for i in range(len(class_names)):
        if len(np.unique(y_true_binarized[:, i])) > 1:  # Check if class exists
            auc = roc_auc_score(y_true_binarized[:, i], y_proba[:, i])
            auc_scores.append(auc)
    
    # Create detailed report
    results = {
        'accuracy': accuracy,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_weighted,
        'mean_auc': np.mean(auc_scores) if auc_scores else 0,
        'per_class_metrics': {
            class_names[i]: {
                'precision': precision[i],
                'recall': recall[i],
                'f1': f1[i],
                'support': support[i]
            } for i in range(len(class_names))
        }
    }
    
    return results

# Use the comprehensive evaluation function
try:
    if 'model' in locals() and 'test_loader' in locals():
        print("\nDetaylı model değerlendirmesi...")
        detailed_results = comprehensive_evaluation(model, test_loader, device, le.classes_)
        
        print(f"\nDetaylı Sonuçlar:")
        print(f"Accuracy: {detailed_results['accuracy']:.4f}")
        print(f"Macro F1: {detailed_results['macro_f1']:.4f}")
        print(f"Weighted F1: {detailed_results['weighted_f1']:.4f}")
        print(f"Mean AUC: {detailed_results['mean_auc']:.4f}")
        
        print("\nSınıf bazında detaylar:")
        for class_name, metrics in detailed_results['per_class_metrics'].items():
            print(f"{class_name}: F1={metrics['f1']:.3f}, Precision={metrics['precision']:.3f}, Recall={metrics['recall']:.3f}")
    else:
        print("Model henüz eğitilmemiş. Önce modeli eğitin.")
except NameError:
    print("Model bulunamadı. Önce modeli eğitin.")

## Model Değerlendirmesi ve İleriye Dönük Çalışmalar

Müzik türü sınıflandırma modelimiz veriyi dengeledikten sonra eğitilmiştir. Sonuçlar değerlendirilirken şunlar göz önünde bulundurulmalıdır:

1. **Veri Kalitesi**: FMA veri setindeki özellikler, ses dosyalarından çıkarılmış özelliklerdir. Daha iyi sonuçlar için ham ses verileri üzerinde spektrogram analizi yapılabilir.

2. **Model Mimarisi**: LSTM modeli, sıralı verilerde başarılı olmasına rağmen, müzik türü tanıma için CNN (Convolutional Neural Network) veya CNN-LSTM hibrit modeller de kullanılabilir.

3. **Hiperparametreler**: Farklı hiperparametreler (örn. öğrenme oranı, katman sayısı, nöron sayısı) ile model performansı artırılabilir.

4. **Veri Dengeleme**: Kullandığımız veri dengeleme yöntemleri, eğitim setindeki sınıf dağılımını eşitlemeye yardımcı olur, ancak sentetik veri oluşturma riskleri de taşır.

5. **Özellik Seçimi**: K-Best algoritması ile seçilen özellikler, modelin daha iyi genelleme yapmasına ve aşırı öğrenmesinin azalmasına yardımcı olabilir. Farklı K değerleri denenerek optimum özellik sayısı bulunabilir.

İleriye dönük çalışmalarda, daha karmaşık modeller, farklı özellik çıkarma teknikleri ve daha büyük veri setleri kullanılarak performans artırılabilir.

## Model Optimizasyonu ve Sorun Giderme

Bu bölüm, model performansını artırmak için çeşitli optimizasyon teknikleri ve sorun giderme yöntemlerini içerir.

In [ ]:
# Model Performans Analizi ve İyileştirme Önerileri

def analyze_model_performance():
    """
    Model performansını analiz et ve iyileştirme önerileri sun
    """
    try:
        if 'detailed_results' in locals() or 'detailed_results' in globals():
            results = detailed_results
            
            print("\n=== MODEL PERFORMANS ANALİZİ ===")
            print(f"Genel Doğruluk: {results['accuracy']:.4f}")
            print(f"Macro F1 Skoru: {results['macro_f1']:.4f}")
            print(f"Weighted F1 Skoru: {results['weighted_f1']:.4f}")
            print(f"Ortalama AUC: {results['mean_auc']:.4f}")
            
            # Performans değerlendirmesi ve öneriler
            if results['accuracy'] < 0.6:
                print("\n⚠️  DÜŞÜK PERFORMANS TESPİT EDİLDİ")
                print("Öneriler:")
                print("1. Daha fazla veri toplama")
                print("2. Farklı model mimarisi deneme (CNN, Transformer)")
                print("3. Hiperparametre optimizasyonu")
                print("4. Veri ön işleme tekniklerini gözden geçirme")
                
            elif results['accuracy'] < 0.75:
                print("\n📊 ORTA SEVİYE PERFORMANS")
                print("İyileştirme önerileri:")
                print("1. Özellik mühendisliği uygulama")
                print("2. Model ensemble teknikleri")
                print("3. Daha sofistike veri dengeleme")
                print("4. Regularization teknikleri")
                
            else:
                print("\n✅ İYİ PERFORMANS")
                print("Model başarılı bir şekilde çalışıyor.")
                
            # Sınıf bazında performans analizi
            print("\n=== SINIF BAZINDA PERFORMANS ===")
            poor_classes = []
            for class_name, metrics in results['per_class_metrics'].items():
                if metrics['f1'] < 0.5:
                    poor_classes.append(class_name)
                    
            if poor_classes:
                print(f"Düşük performanslı sınıflar: {', '.join(poor_classes)}")
                print("Bu sınıflar için:")
                print("- Daha fazla veri toplama")
                print("- Sınıf ağırlıklandırma")
                print("- Focal loss kullanma")
                
        else:
            print("Model değerlendirmesi henüz yapılmamış.")
            
    except Exception as e:
        print(f"Performans analizi sırasında hata: {e}")

def get_improvement_suggestions():
    """
    Gelişmiş iyileştirme önerileri
    """
    suggestions = {
        "Veri İyileştirmeleri": [
            "Veri temizleme ve outlier detection",
            "Feature scaling yöntemlerini karşılaştırma (RobustScaler, MinMaxScaler)",
            "Veri artırma teknikleri (audio augmentation)"
        ],
        "Model İyileştirmeleri": [
            "Bidirectional LSTM kullanma",
            "Attention mechanism ekleme",
            "Residual connections",
            "Batch normalization optimizasyonu"
        ],
        "Eğitim İyileştirmeleri": [
            "Learning rate scheduling",
            "Gradient clipping",
            "Warm-up strategies",
            "Cyclical learning rates"
        ],
        "Ensemble Yöntemleri": [
            "Farklı model mimarilerini birleştirme",
            "Voting classifiers",
            "Stacking",
            "Bagging"
        ]
    }
    
    print("\n=== GELİŞMİŞ İYİLEŞTİRME ÖNERİLERİ ===")
    for category, items in suggestions.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

# Performans analizini çalıştır
analyze_model_performance()
get_improvement_suggestions()

print("\n" + "="*80)
print("MODEL EĞİTİMİ VE DEĞERLENDİRMESİ TAMAMLANDI")
print("="*80)

In [ ]:
# SIMPLIFICATION: Removed cross-validation and hyperparameter tuning
# These add complexity and can lead to overfitting
print("Cross-validation and hyperparameter tuning removed for simplicity.")
print("Focus on getting basic model to generalize well first.")
print("Once basic performance is good, then optimize hyperparameters.")

In [ ]:
# FURTHER SIMPLIFICATION: Remove complex hyperparameter tuning
print("Complex hyperparameter tuning removed to prevent overfitting.")
print("Current model uses conservative parameters proven to work well.")
print("Focus on data quality and model architecture before fine-tuning.")

In [ ]:
# TRAINING DIAGNOSTICS - Check what's happening during training
print("\n=== TRAINING DIAGNOSTICS ===")

# Quick check of first batch performance
with torch.no_grad():
    # Get first batch
    first_batch = next(iter(train_loader))
    inputs, labels = first_batch[0].to(device), first_batch[1].to(device)
    
    # Check model output before training
    model.eval()
    outputs = model(inputs)
    _, predictions = torch.max(outputs, 1)
    
    print(f"First batch:")
    print(f"  Input shape: {inputs.shape}")
    print(f"  Labels shape: {labels.shape}")
    print(f"  Output shape: {outputs.shape}")
    print(f"  Unique labels in batch: {torch.unique(labels)}")
    print(f"  Unique predictions: {torch.unique(predictions)}")
    
    # Check if model is outputting reasonable probabilities
    probs = torch.softmax(outputs, dim=1)
    print(f"  Output probabilities - Min: {probs.min():.4f}, Max: {probs.max():.4f}")
    print(f"  Most confident prediction: {probs.max(dim=1)[0].mean():.4f}")
    
    # Check accuracy on first batch
    accuracy = (predictions == labels).float().mean()
    print(f"  Random accuracy on first batch: {accuracy:.4f}")
    print(f"  Expected random accuracy: {1/num_classes:.4f}")
    
    if accuracy < 0.05:  # Much worse than random
        print("  ⚠️  WARNING: Model performing much worse than random!")
        print("  This suggests a serious issue with model or data.")

print("\n" + "="*50)
print("Ready to start training. Watch for:")
print("1. Training accuracy should improve from random levels")
print("2. Validation accuracy should follow training (with some gap)")
print("3. Loss should decrease steadily")
print("4. If validation accuracy stays near random, there's an issue")
print("="*50)

In [ ]:
# FINAL ACCURACY BOOST WITH BEST PRACTICES
print("\n=== FINAL PERFORMANCE OPTIMIZATION ===")

# Option 1: Train a single optimized model with all improvements
print("Training optimized single model...")
optimized_model = MusicGenreLSTM(input_size, hidden_size, num_layers, num_classes, dropout).to(device)

# Enhanced optimizer settings
optimizer_opt = optim.AdamW(optimized_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler_opt = optim.lr_scheduler.OneCycleLR(
    optimizer_opt, max_lr=0.003, epochs=25, steps_per_epoch=len(train_loader)
)

# Custom training function with OneCycleLR
def train_optimized_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0
    PATIENCE = 8
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()  # Step after each batch for OneCycleLR
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader.dataset)
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_train_acc = train_correct / train_total
        epoch_val_acc = val_correct / val_total
        
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store metrics
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_accs.append(epoch_train_acc)
        val_accs.append(epoch_val_acc)
        
        print(f'Epoch {epoch+1}/{num_epochs} - '
              f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, '
              f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}, LR: {current_lr:.6f}')
        
        # Save best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_model = model.state_dict().copy()
            patience_counter = 0
            print(f'*** New best validation accuracy: {best_val_acc:.4f} ***')
        else:
            patience_counter += 1
            
        if patience_counter >= PATIENCE:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    # Load best model
    model.load_state_dict(best_model)
    print(f'Final optimized model validation accuracy: {best_val_acc:.4f}')
    return model, train_losses, val_losses, train_accs, val_accs

# Train the optimized model
print("\nTraining final optimized model...")
final_model, final_train_losses, final_val_losses, final_train_accs, final_val_accs = train_optimized_model(
    optimized_model, train_loader, val_loader, criterion, optimizer_opt, scheduler_opt, num_epochs=25
)

print("\n🎯 FINAL MODEL TRAINING COMPLETED!")
print("Ready for test evaluation...")
print("Expected improvement: 61% → 65-70% test accuracy")

In [ ]:
# FINAL TEST EVALUATION
print("\n=== FINAL TEST EVALUATION ===")

# Evaluate the final optimized model
def evaluate_final_model(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    y_proba = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_proba.extend(probabilities.cpu().numpy())
    
    # Calculate accuracy
    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    
    return accuracy, y_true, y_pred, np.array(y_proba)

# Test the final model
final_accuracy, y_true_final, y_pred_final, y_proba_final = evaluate_final_model(final_model, test_loader, device)

print(f"\n🎉 FINAL TEST ACCURACY: {final_accuracy:.4f} ({final_accuracy*100:.1f}%)")

# Compare with baseline
baseline_accuracy = 0.61  # Previous best
improvement = final_accuracy - baseline_accuracy
print(f"Improvement over baseline: {improvement:.4f} ({improvement*100:.1f} percentage points)")

if final_accuracy > 0.65:
    print("🚀 EXCELLENT! Achieved target of >65% accuracy!")
elif final_accuracy > baseline_accuracy:
    print("✅ GOOD! Model improved over baseline.")
else:
    print("📊 Performance maintained at baseline level.")

# Quick class-wise performance
from sklearn.metrics import classification_report
print("\nClass-wise Performance:")
report = classification_report(y_true_final, y_pred_final, target_names=le.classes_, output_dict=True, zero_division=0)
for class_name, metrics in report.items():
    if isinstance(metrics, dict) and 'f1-score' in metrics:
        print(f"{class_name}: F1={metrics['f1-score']:.3f}")

print(f"\nMacro F1: {report['macro avg']['f1-score']:.4f}")
print(f"Weighted F1: {report['weighted avg']['f1-score']:.4f}")

print("\n" + "="*60)
print("🎯 MUSIC GENRE CLASSIFICATION OPTIMIZATION COMPLETE!")
print("="*60)

## 🚀 Performance Optimization Summary

### Improvements Applied:

1. **Enhanced Model Architecture:**
   - Bidirectional LSTM for better feature capture
   - Optimized batch normalization and dropout
   - Two-layer classifier for better decision boundaries

2. **Advanced Training Techniques:**
   - OneCycleLR scheduler for better convergence
   - Label smoothing to prevent overconfidence
   - Gradient clipping for training stability
   - AdamW optimizer with weight decay

3. **Data Enhancement:**
   - Increased K-best features to 150
   - Simple data augmentation with noise injection
   - Improved sequence length for temporal modeling

4. **Training Optimization:**
   - Balanced batch size (256)
   - Appropriate early stopping patience
   - Best model selection based on validation accuracy

### Expected Results:
- **Baseline:** 61% test accuracy
- **Target:** 65-70% test accuracy
- **Key Improvement:** Better generalization without overfitting

### Next Steps if Needed:
- Ensemble methods (3-5 models)
- Advanced augmentation techniques
- Hyperparameter fine-tuning
- Architecture search (CNN, Transformer)

In [ ]:
# IMMEDIATE TRAINING VERIFICATION
print("\n=== QUICK TRAINING VERIFICATION ===")
print("Testing if the enhanced oversampling actually improves learning...")

# Quick check of first few batches to see class distribution
print("\nChecking class distribution in first few training batches:")
class_counter = Counter()
for i, (inputs, labels) in enumerate(train_loader):
    for label in labels:
        class_counter[label.item()] += 1
    if i >= 2:  # Check first 3 batches
        break

print("Classes seen in first 3 batches:")
for class_idx, count in sorted(class_counter.items()):
    if class_idx < len(le.classes_):
        print(f"  {le.classes_[class_idx]}: {count} samples")

total_seen = sum(class_counter.values())
if len(class_counter) < len(le.classes_) * 0.7:  # Less than 70% of classes
    print(f"\n⚠️  WARNING: Only {len(class_counter)} out of {len(le.classes_)} classes seen")
    print("This suggests severe class imbalance still exists!")
else:
    print(f"\n✅ Good diversity: {len(class_counter)} out of {len(le.classes_)} classes seen")

print(f"\nReady to train with {len(y_train_tensor)} total training samples")
print(f"Expected accuracy improvement from better balance: 63% → 66-68%")
print("="*60)